In [1]:
import warnings, os
warnings.filterwarnings("ignore")

from copy import copy
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, accuracy_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef, precision_score, recall_score, f1_score


import seaborn as sns
import matplotlib.pyplot as plt
from joblib import dump, load

from aggmap import AggMap, loadmap
from aggmap import AggMapNet as AggModel

from aggmap.AggMapNet import load_model, save_model
from aggmap import show


np.random.seed(666) #just for reaptable results


def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_pred

    '''
    the metrics are taken from orignal paper:
    Meta-Signer: Metagenomic Signature Identifier based on Rank Aggregation of Features
    https://github.com/YDaiLab/Meta-Signer/blob/bd6a1cd98d1035f848ecb6e53d9ee67a85871db2/src/utils/metasigner_io.py#L34
    '''
    auc = roc_auc_score(y_true, y_score, average='weighted')        
    mcc = matthews_corrcoef(y_true, y_pred)
    pres = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    acc = accuracy_score(y_true, y_pred)
    
    print('roc-auc: %.3f, mcc: %.3f, pres: %.3f, recall: %.3f, f1: %.3f' % (auc, mcc, pres, recall, f1))

    return acc, auc, mcc, pres, recall, f1

2022-10-03 17:34:24.393442: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# read data

In [2]:
task = 'IBD'
data_path = '../../../01_data/species_level/%s/' % (task)
save_dir = '%s_results' % task
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

dfa = pd.read_csv(os.path.join(data_path, 'abundance.tsv'),sep='\t', header=None, index_col=0)
dfy = pd.read_csv(os.path.join(data_path, 'labels.txt'),sep='\t', header=None)
dfx = dfa.T
dfy = pd.get_dummies(dfy[0].map({'ibd':1, 'n':0}))
Y = dfy.values

# generate Fmaps

In [3]:
#could be optimized
featHPs = {"best_fill":1e-2, "best_scale_method":'minmax', "best_channel_number":5}
dfx = np.log(dfx + featHPs.get('best_fill'))
mp = AggMap(dfx, metric = 'correlation')
mp = mp.fit(cluster_channels = featHPs.get('best_channel_number'), verbose = 0, var_thr = 0)
X = mp.batch_transform(dfx.values, scale = featHPs.get('best_scale_method')) # NaN values should be the lowest value
mp.plot_grid(save_dir)
mp.plot_scatter(save_dir)
mp.save(os.path.join(save_dir, 'agg.mp'))

2022-10-03 17:34:25,019 - INFO - [bidd-aggmap] - Calculating distance ...
2022-10-03 17:34:25,025 - INFO - [bidd-aggmap] - the number of process is 16


100%|####################################################################################################################################| 443/443 [00:00<00:00, 6030.92it/s]


2022-10-03 17:34:27,433 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2022-10-03 17:34:29,738 - INFO - [bidd-aggmap] - Applying grid assignment of feature points, this may take several minutes(1~30 min)
2022-10-03 17:34:29,932 - INFO - [bidd-aggmap] - Finished


100%|######################################################################################################################################| 110/110 [00:01<00:00, 58.16it/s]


2022-10-03 17:34:31,902 - INFO - [bidd-aggmap] - generate file: IBD_results/feature points_443_correlation_umap_mp
2022-10-03 17:34:31,909 - INFO - [bidd-aggmap] - save html file to IBD_results/feature points_443_correlation_umap_mp
2022-10-03 17:34:31,909 - INFO - [bidd-aggmap] - generate file: IBD_results/feature points_443_correlation_umap_scatter
2022-10-03 17:34:31,914 - INFO - [bidd-aggmap] - save html file to IBD_results/feature points_443_correlation_umap_scatter


['IBD_results/agg.mp']

# 10FCV

In [4]:
gpuid = 1

outer_fold = 10
repeat_seeds = [8, 16, 32, 64, 128, 256, 1024, 2048, 4096, 8192] #10 repeats random seeds 8, 16, 32, 64, 128

each_fold_results = []
run_all_res = []

for i, repeat_seed in enumerate(repeat_seeds): 
    outer = StratifiedKFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(range(len(dfy)), dfy.idxmax(axis=1))
    run_one_res = []
    for j, idx in enumerate(outer_idx):
        fold_num = "fold_%s" % str(j).zfill(2) 
        print('#'*50 + ' repeat_seed: %s; %s ' % (repeat_seed, fold_num) + '#'*50 )
        
        train_idx, test_idx = idx

        testY = Y[test_idx]
        testX = X[test_idx]
        
        trainX = X[train_idx]
        trainY = Y[train_idx]

        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))

        clf = AggModel.MultiClassEstimator(epochs = 50,  batch_size = 2, verbose = 0, gpuid=gpuid) #
        clf.fit(trainX, trainY)  #, 
        
        ## save model for explaination
        if i == 0:
            clf.save_model(os.path.join(save_dir, '%s.model' % fold_num))
            paras = clf.get_params()
            paras.update({'featHPs':featHPs})
            pd.Series(paras).to_json(os.path.join(save_dir, 'HPs.json'))

        pred_proba = clf.predict_proba(testX)
        y_true = testY[:,1] 
        y_score = pred_proba[:,1]
        y_pred = np.argmax(pred_proba, axis=1)
        
        dfr = pd.DataFrame([y_true, y_score, y_pred]).T
        dfr.columns = ['y_true', 'y_score', 'y_pred']
        dfr.index = dfy.iloc[test_idx].index
        
        acc, auc, mcc, pres, recall, f1  = score(dfr)
        run_one_res.append(dfr)
        ts = pd.Series([acc, auc, mcc, pres, recall, f1, i, repeat_seed]).round(3)
        ts.index = ['acc','auc', 'mcc', 'pres', 'recall', 'f1', 'i', 'repeat_seed']

        print(ts.to_dict())
        each_fold_results.append(ts.to_dict())
    run_all_res.append(pd.concat(run_one_res))

2022-10-03 17:34:31.963038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


################################################## repeat_seed: 8; fold_00 ##################################################

 input train and test X shape is (99, 22, 21, 5), (11, 22, 21, 5) 
MultiClassEstimator(batch_norm=False, batch_size=2, conv1_kernel_size=13,
                    dense_avf='relu', dense_layers=[128], dropout=0.0,
                    epochs=50, gpuid='1', last_avf='softmax',
                    loss='categorical_crossentropy', lr=0.0001, metric='ACC',
                    monitor='val_loss', n_inception=2,
                    name='AggMap MultiClass Estimator', patience=10000,
                    random_state=32, verbose=0)


2022-10-03 17:34:31.979550: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 17:34:31.979660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 17:34:31.979891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 17:34:31.980460: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

saving model to IBD_results/fold_00.model
roc-auc: 0.889, mcc: 0.671, pres: 0.918, recall: 0.909, f1: 0.896
{'acc': 0.909, 'auc': 0.889, 'mcc': 0.671, 'pres': 0.918, 'recall': 0.909, 'f1': 0.896, 'i': 0.0, 'repeat_seed': 8.0}
################################################## repeat_seed: 8; fold_01 ##################################################

 input train and test X shape is (99, 22, 21, 5), (11, 22, 21, 5) 
MultiClassEstimator(batch_norm=False, batch_size=2, conv1_kernel_size=13,
                    dense_avf='relu', dense_layers=[128], dropout=0.0,
                    epochs=50, gpuid='1', last_avf='softmax',
                    loss='categorical_crossentropy', lr=0.0001, metric='ACC',
                    monitor='val_loss', n_inception=2,
                    name='AggMap MultiClass Estimator', patience=10000,
                    random_state=32, verbose=0)
saving model to IBD_results/fold_01.model
roc-auc: 0.944, mcc: 0.671, pres: 0.918, recall: 0.909, f1: 0.896
{'acc': 0.90

In [5]:
pd.DataFrame(each_fold_results).groupby('repeat_seed').mean().mean()

acc       0.87533
auc       0.93760
mcc       0.60170
pres      0.86520
recall    0.87533
f1        0.85739
i         4.50000
dtype: float64

In [6]:
pd.DataFrame(each_fold_results).groupby('repeat_seed').std().mean()

acc       0.088806
auc       0.085647
mcc       0.314170
pres      0.125726
recall    0.088806
f1        0.109032
i         0.000000
dtype: float64

In [7]:
pd.DataFrame(each_fold_results).to_csv(os.path.join(save_dir, 'performance_results.csv'))

# feature importance

In [8]:
all_imps = []
for i in range(10):
    clf = load_model(os.path.join(save_dir, 'fold_%s.model' % str(i).zfill(2)))
    dfe = clf.explain_model(mp, clf.X_, clf.y_, binary_task=True, apply_logrithm=False)
    df_imp = dfe.col_1_importance.to_frame(name = 'fold_%s_imp' % str(i).zfill(2))
    all_imps.append(df_imp)

4/4 [==============================] - 0s 1ms/step
calculating feature importance for column 1 ...


  0%|                                                                                                                                                | 0/462 [00:00<?, ?it/s]

65/65 [==============================] - 0s 2ms/step


  5%|######1                                                                                                                                | 21/462 [00:00<00:04, 93.97it/s]

62/62 [==============================] - 0s 2ms/step


...

 95%|###############################################################################################################################9      | 441/462 [00:04<00:00, 91.75it/s]

62/62 [==============================] - 0s 2ms/step


100%|#####################################################################################################################################7| 461/462 [00:05<00:00, 92.39it/s]

4/4 [==============================] - 0s 1ms/step


100%|######################################################################################################################################| 462/462 [00:05<00:00, 89.31it/s]


In [9]:
dfi = dfe[dfe.columns[:-1]]
dfi['scatter_x'] = dfi.v.map(mp.df_embedding.x)
dfi['scatter_y'] = dfi.v.map(mp.df_embedding.y)

dfimp_all = pd.concat(all_imps, axis=1)
dfi = dfi.join(dfimp_all.mean(axis=1).to_frame(name = 'avg_imp'))
dfi = dfi.join(dfimp_all)
dfi.to_csv(os.path.join(save_dir, 'feature_imp_results.csv'))

In [10]:
dfi

,x,y,v,Subtypes,colors,scatter_x,scatter_y,avg_imp,fold_00_imp,fold_01_imp,fold_02_imp,fold_03_imp,fold_04_imp,fold_05_imp,fold_06_imp,fold_07_imp,fold_08_imp,fold_09_imp
0,0,0,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,cluster_04,#0010ff,1.919993,0.598925,-0.311019,-0.341685,-0.302067,-0.362262,-0.286650,-0.337547,-0.315830,-0.258773,-0.317028,-0.334428,-0.253924
1,1,0,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,cluster_04,#0010ff,1.943752,0.648995,-0.334407,-0.367489,-0.331313,-0.382972,-0.314872,-0.370340,-0.344244,-0.271312,-0.318480,-0.364958,-0.278094
2,2,0,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,cluster_04,#0010ff,1.966278,0.640597,-0.334696,-0.367936,-0.331622,-0.383361,-0.315087,-0.370959,-0.344438,-0.271585,-0.318728,-0.364985,-0.278263
3,3,0,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,cluster_04,#0010ff,1.972043,0.634254,-0.334659,-0.367822,-0.331555,-0.383311,-0.315041,-0.370942,-0.344439,-0.271577,-0.318718,-0.364920,-0.278263
4,4,0,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,cluster_04,#0010ff,1.985219,0.683424,-0.334712,-0.367959,-0.331644,-0.383394,-0.315059,-0.370967,-0.344447,-0.271627,-0.318726,-0.365035,-0.278263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,16,21,NaN-457,NaN,#000000,NaN,NaN,-0.334732,-0.368001,-0.331665,-0.383420,-0.315095,-0.370982,-0.344481,-0.271640,-0.318738,-0.365035,-0.278263
458,17,21,NaN-458,NaN,#000000,NaN,NaN,-0.334732,-0.368001,-0.331665,-0.383420,-0.315095,-0.370982,-0.344481,-0.271640,-0.318738,-0.365035,-0.278263
459,18,21,NaN-459,NaN,#000000,NaN,NaN,-0.334732,-0.368001,-0.331665,-0.383420,-0.315095,-0.370982,-0.344481,-0.271640,-0.318738,-0.365035,-0.278263
460,19,21,NaN-460,NaN,#000000,NaN,NaN,-0.334732,-0.368001,-0.331665,-0.383420,-0.315095,-0.370982,-0.344481,-0.271640,-0.318738,-0.365035,-0.278263
